--------------
**Author**: Gunnvant

**Description**: Demo notebook to show how logging happens

--------------

In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [2]:
x_path="s3://gunmlartifacts/feature_store/X.npy"
y_path="s3://gunmlartifacts/feature_store/y.npy"

In [3]:
from s3fs.core import S3FileSystem
import numpy as np
def get_s3(s3_path):
    s3 = S3FileSystem()
    return np.load(s3.open(s3_path), allow_pickle=True)

In [4]:
X = get_s3(x_path)
y = get_s3(y_path)

In [5]:
X_train,X_test,y_train,y_test = train_test_split(X,y,train_size=0.80)

In [6]:
import mlflow

In [7]:
mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [8]:
mlflow.set_experiment("hotel_occupancy")

2023/09/12 10:14:25 INFO mlflow.tracking.fluent: Experiment with name 'hotel_occupancy' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://gunmlartifacts/project1/1', creation_time=1694493865796, experiment_id='1', last_update_time=1694493865796, lifecycle_stage='active', name='hotel_occupancy', tags={}>

In [9]:
params = {"n_estimators":120,"max_depth":20,"n_jobs":-1}
clf = RandomForestClassifier(**params)

In [10]:
from sklearn.metrics import roc_auc_score, confusion_matrix

In [11]:
with mlflow.start_run():
    clf = clf.fit(X_train,y_train)
    preds = clf.predict(X_test)
    pred_proba = clf.predict_proba(X_test)[:,1]
    cm = confusion_matrix(y_test,preds)
    auc=roc_auc_score(y_test,pred_proba)
    t_n, f_p, f_n, t_p = cm.ravel()
    mlflow.log_metrics({"tn":t_n,'fp':f_p,'fn':f_n,'tp':t_p,"auc":auc})
    mlflow.sklearn.log_model(clf,'rf_model',input_example=X_test[0:,])
    mlflow.log_params(params)